In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df= pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df.isna().count()

In [ ]:
cols = df.columns
num_cols = df._get_numeric_data().columns
char_cols=list(set(cols) - set(num_cols))
char_cols

In [ ]:
from sklearn.impute import SimpleImputer
si= SimpleImputer()
def impute(df):
    df[num_cols]=si.fit_transform(df[num_cols])
    return df
def sub(df):
    for i in char_cols:
        for j in range(len(df[i])):
            if df[i][j] is '' or df[i][j] is 'NaN':
                df[i][j]='NA'
    return df
impute(df)
sub(df)
        

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
def encode(df):
    for i in char_cols:
        df[i]= le.fit_transform(df[i])
    return df
encode(df)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
df.iloc[:,:-1]=ss.fit_transform(df.iloc[:,:-1])

In [ ]:
df

In [ ]:
!pip install auto-sklearn

In [ ]:
from sklearn.model_selection import train_test_split
import autosklearn.classification
X= df.iloc[:,:-1]
y= df.iloc[:,-1]
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,shuffle=True,stratify=df[['stroke']])


In [ ]:
model = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=5*60, per_run_time_limit=30, n_jobs=8)
model.fit(X_train, y_train)

In [ ]:
print(model.sprint_statistics())

In [ ]:
from sklearn.metrics import accuracy_score
y_hat = model.predict(X_test)
acc = accuracy_score(y_test, y_hat)
print("Accuracy: %.3f" % acc)

In [ ]:
!pip install openpyxl

In [ ]:
dfk= X_test.iloc[:,:-1]
dfk['Predicted']= y_hat
data = dfk.to_excel("outputautoML.xlsx",index=False,header=True)